In [1]:
import time
import random
from PyOCR import *
from Summarizer import *
import subprocess
from os import listdir
from os.path import isfile, join
import os.path
import sklearn.datasets
import nltk.stem
import numpy as np
import pandas as pd
from stop_words import get_stop_words
import re
import scipy as sp
import PyPDF2
import docx
import sys
import os
import os.path
import string
import docx
import docx2txt
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
stop_words = get_stop_words('english')
english_stemmer = nltk.stem.SnowballStemmer('english')

In [4]:
def file_operation():
    data = []
    mypath = "Users/sahebsingh/Desktop/Projects/Honeywell/Honeywell" #Path to folder/Directory
    
    list_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for k in range(len(list_files)):
        document = docx.Document(list_files[k])
    print (document.paragraphs)

In [5]:
def getPdf(file):
    data = []
    file = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(file)
    page_number = pdfReader.numPages
    for i in range(0, page_number):
        pageObj = pdfReader.getPage(i)
        data.append(pageObj.extractText())
        
    return '\n'.join(data)
    

In [6]:
def getDocx(filename):
    my_text = docx2txt.process(filename)
    
    return np.array(my_text)
    

In [7]:
data1 = getDocx('2.docx')

In [8]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
    
    

In [9]:
data4 = getPdf('7.pdf')

In [10]:
import nltk
# nltk.download('punkt')
# nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest


def summarize(text, n):
    
    sents = sent_tokenize(text)
    #assert n <= len(sents)
    if(n>=len(sents)):
        n = random.randint(int(len(sents)/2), len(sents))
    
    wordSent = word_tokenize(text.lower())
    stopWords = set(stopwords.words('english')+list(punctuation))
    
    wordSent= [word for word in wordSent if word not in stopWords]
    freq = FreqDist(wordSent)
    ranking = defaultdict(int)
    
    for i, sent in enumerate(sents):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i] += freq[w]
    sentsIDX = nlargest(n, ranking, key=ranking.get)
    print('Summarise Completed')
    return [sents[j] for j in sorted(sentsIDX)]

summaryArr = summarize(data4, 20)
# summaryArr

Summarise Completed


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
def Vect(data, T):
    
    vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')
    X = vectorizer.fit_transform(summaryArr)
    km = KMeans(n_clusters = T, init = 'k-means++', max_iter = 100, n_init = 2, verbose = True)
    km.fit(X)
    np.unique(km.labels_, return_counts=True)
    text={}
    for i,cluster in enumerate(km.labels_):
        oneDocument = summaryArr[i]
        if cluster not in text.keys():
            text[cluster] = oneDocument
        else:
            text[cluster] += oneDocument
    stopWords = set(stopwords.words('english')+list(punctuation))
    keywords = {}
    counts={}
    for cluster in range(T):
        word_sent = word_tokenize(text[cluster].lower())
        word_sent=[word for word in word_sent if word not in stopWords]
        freq = FreqDist(word_sent)
        keywords[cluster] = nlargest(100, freq, key=freq.get)
        counts[cluster]=freq
        uniqueKeys={}
    for cluster in range(T):   
        other_clusters=list(set(range(T))-set([cluster]))
        keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
        unique=set(keywords[cluster])-keys_other_clusters
        uniqueKeys[cluster]=nlargest(10, unique, key=counts[cluster].get)
    #print(uniqueKeys)
    print(" ")
    print(" ")
    print('Vect Completed')
    return [X, km, vectorizer]
Vect(data4, 10)

Initialization complete
Iteration  0, inertia 7.020
Iteration  1, inertia 3.534
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
Initialization complete
Iteration  0, inertia 7.292
Iteration  1, inertia 4.016
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
 
 
Vect Completed


[<20x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 480 stored elements in Compressed Sparse Row format>,
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
     n_clusters=10, n_init=2, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=True),
 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.5, max_features=None, min_df=2,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words='english', strip_accents=None, sublinear_tf=False,
         token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
         vocabulary=None)]

In [14]:
Z = Vect(data4, 10)
X = Z[0]
km = Z[1]
vectorizer = Z[2]

Initialization complete
Iteration  0, inertia 7.030
Iteration  1, inertia 4.074
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
Initialization complete
Iteration  0, inertia 7.146
Iteration  1, inertia 3.703
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
 
 
Vect Completed


In [15]:
post = """ his document is to provide an example of a weight and balance document for an aircraft type certificate application in accordance with CS-LSA. The document can be used even if the applicant does not own a DOA. It does not substitute, in any of its parts, the prescriptions of Part-21 and its amendments.
This document is intended to assist applicants in applying for an LSA RTC/TC and therefore demonstrating compliance of the design to the requirements.
The document should not be read as a template and it should not be used as a form to fill. The content shall be checked for appropriateness and changed accordingly by the applicant.
The required information can be presented entirely in this document, or in additional documents appropriately identified and referred to.
"""

In [16]:
new_post_vec = vectorizer.transform([post])
print(new_post_vec)
#new_post_label = km.predict(new_post_vec)[0]#We will predict it's cluster 

#similar_indices = (km.labels_ == new_post_label).nonzero()[0]


  (0, 118)	0.177560834545
  (0, 115)	0.214809079162
  (0, 112)	0.164266360598
  (0, 88)	0.328532721196
  (0, 58)	0.134700765553
  (0, 50)	0.134700765553
  (0, 49)	0.762108695881
  (0, 46)	0.143289237078
  (0, 44)	0.164266360598
  (0, 40)	0.177560834545
  (0, 29)	0.134700765553
  (0, 27)	0.214809079162
  (0, 23)	0.164266360598


In [17]:
# Old Method, Not Doing that anymore
similar = []
for i in similar_indices:
    print(X[i])
    dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
    similar.append((dist, summaryArr[i]))


NameError: name 'similar_indices' is not defined

In [18]:
# New Method
similar = []
for i in range(0,20):
    dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
    similar.append((dist, summaryArr[i]))

In [19]:
similar = sorted(similar) 
print(len(similar))
show_at_1 = similar[0]
show_at_2 = similar[1]
show_at_3 = similar[2]

20


In [20]:
print("=== #1 ===")
print(show_at_1)
print()


=== #1 ===
(1.2409373376568382, '!4.Requirements  \nThis document covers the following certification specifications requirements: \nTable 1\n Ð Requirements  \n5.Reference for centre of gravity values \nDatum plane:  leading edge at wing mean aerodynamic chord \nRequirement \nCS-LSA\n.15, 29th \nJuly 2013 \namendment 1 (ASTM F2245-12d)\nSubject of requirement\nReferenced \nchapter\n3.1Definitions:\nÐ3.1.2maximum empty weight\n, W\nE Ð largest empty weight of the \nairplane, including all operational equipment that is \ninstalled in the airplane: weight of the airframe, \npowerplant, required equipment, optional and specific \nequipment, fixed ballast, full engine coolant and oil, \nhydraulic fluid, and the unusable fuel.')



In [21]:
print("=== #2 ===")
print(show_at_2)
print()



=== #2 ===
(1.2541249575106446, '6.34.2.3Empty CG, most forward, and most rearward CG shall be \ndetermined.7.1 7.25.2Flight Loads:\n5.2.1.3Maximum Zero Wing Fuel Weight, W\nZWF Ð The maximum \nallowable weight of the airplane without any fuel in the \nwing tank(s) must be established if it is less than maximum \ndesign weight, W.\n6.5\nEASA Example Documents\nWEIGHT AND BALANCE \nCompany \nlogo \nDocument reference:\n company \n!namePage \n  6of 12DateRevision\n Doc.-No.')



In [22]:
print("=== #3 ===")
print(show_at_3)



=== #3 ===
(1.2627154915437777, 'Compliance has been shpow in \nchapter 6.2.Requireme\nnt reference\nSubject\nCS-LSA \nF2245-12d \n4.2.1 4.2.1.1 4.2.1.24.2.1 The minimum useful load, W\nU, shall be equal to or greater than the sum of: \n4.2.1.1 An occupant weight of 845 N (190 lb) for each occupant seat in aircraft, plus \n4.2.1.2 The weight of consumable substances, such as fuel, as required for a 1-h \nflight at V\nH. Consumption rates must be based on test results for the specific \napplication.')


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
articleURL = "http://curia.europa.eu/juris/document/document.jsf?text=&docid=139407&pageIndex=0&doclang=EN&mode=lst&dir=&occ=first&part=1&cid=52454"
def getText(url):
    page = urlopen(url).read().decode('utf8', 'ignore')
    soup = BeautifulSoup(page, 'lxml')
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text.encode('ascii', errors='replace').decode().replace("?","")
text = getText(articleURL)

In [23]:
def process(text, n):
    text1 = summarize(text, n) 
    Z = Vect(text1, T)
    X = Z[0]
    km = Z[1]
    vectorizer = Z[2]
    new_post_vec = vectorizer.transform([post])
    similar = []
    for i in range(0, 100):
        dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
        similar.append((dist, summaryArr[i]))
    similar = sorted(similar) 
    show_at_1 = similar[0]
    show_at_2 = similar[1]
    show_at_3 = similar[2]
    return [show_at_1, show_at_2, show_at_2]
        
    

In [ ]:
def final(data, post, n = 100):
    Complete = {}
    filename, type1 = os.path.splitext(data)
    if type1 == 'pdf':
        text = getPdf(data)
        Output = process(text)
        
        
    if type1 == ('ppt' or 'pptx'):
        text = #Complete for PPT
        Output = process(text)
    
    for i in range(1, 17):
        text = getPdf(str(i)+'.pdf')
        text1 = summarize(text, n) 
        T = Vect(text1)
        X = T[0]
        km = T[1]
        vectorizer = T[2]
        Complete[str(i)+'.pdf'] = X
        
        
        
        
        
        
    

In [335]:
text = getPdf(str(1)+'.pdf')

In [173]:
    Complete = {}
    for i in range(1, 16):
        print(str(i)+'.pdf')
        text1 = getPdf(str(i)+'.pdf')
        #text = getPdf('')
        text2 = summarize(text1, 10) 
        print(text2)
        Z = Vect(text2, 10)
        X = Z[0]
        km = Z[1]
        vectorizer = Z[2]
        Complete[str(i)+'.pdf'] = X

1.pdf
Summarise Completed
['Instrument  Ground School 2015\nHolding Patterns and ProceduresA holding pattern provides a protected ai\nrspace for a safe operation during the hold.', 'Instrument  Ground School 2015\n\nInstrument  Ground School 2015\nHolding PatternsStandardholding \npattern -all turns made \n\nto right.The inbound legof a holding pattern is 1 minuteat & below 14,000 ft. and 1.5 \nminutes above 14,000 \nft.Outbound leg should be adjustedto yield a 1 \nminute inbound leg.Timing of the outbound leg should begin either abeam \nthe fix or after completing the outbound turn.Each holding pattern has a fix, a direction to hold FROM the fix, and an airway, bearing, course, radial, or route on which the aircraft is to hold.', 'Instrument  Ground School 2015\nHolding PatternsDME leg length may be used \ninstead of time \nfor holding \npatterns.When used, DME leg length \nrefers to the \noutbound leg \nlength.', 'Instrument  Ground School 2015\nSpeed LimitsMaximum holding speeds are

Summarise Completed
['1   Volume 16, Issue 19 November 14, 2008 Volume 16, Issue 19      \n \n \n \n   \n \n \n \n \n   \n \n \n \n  A FOCUS ON SAFETY ALL:\n The Gulfstream manuals now call only for bonding \nthe fuel source to the aircraft during fueling or defueling...2                                   TOP STORIES ALL: \nAn overview of the large-cabin AMM revisions \nTechnical Publications has produced to date in 2008–\n3  \nAstraŽ/SPXŽ/G100\n®/G150®/G200®: Rudder Trim \ninstallations on the Astra through G200 aircraft are \nsusceptible to water ingress into the trim actuator \nhousings...4  G150®: ﬁGEN LOAD UNBALANCEﬂ status messages \nhave been traced to a poor gr\nound for the APU generator \nreturn cable at 10GND...5  G200®: An issue surrounding the operational \ncharacteristics of the Entr\ny Light Switch Panel (CMS1 \nCabin System) has been brought \nto our attention by a \nG200 operator...5  GII®/GIII®: During left engine start, when the start button \nwas depressed, both eng

In [586]:
# Updated Version is Below
"""
for l in range(2, 16):
    print(l)
    X = Complete[str(l)+'.pdf']

    similar = []
    for i in range(1, 10):
        print(i)
        dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
        similar.append((dist, summaryArr[i]))
"""
    

2
1
2
3
4
5
6
7
8
9
3
1
2
3
4
5
6
7
8
9
4
1
2
3
4
5
6
7
8
9
5
1
2
3
4
5
6
7
8
9
6
1
2
3
4
5
6
7
8
9
7
1
2
3
4
5
6
7
8
9
8
1
2
3
4
5
6
7
8
9
9
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
11
1
2
3
4
5
6
7
8
9
12
1
2
3
4
5
6
7
8
9
13
1
2
3
4
5
6
7
8
9
14
1
2
3
4
5
6
7
8
9
15
1
2
3
4
5
6
7
8
9


In [169]:
similar = []
for l in range(1, 16):
    name = str(l)+'.pdf'
    X = Complete[str(l)+'.pdf']
    
    for i in range(1, 10):
        dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
        similar.append((dist, name, text2[i]))

    

In [243]:
similar = sorted(similar) 
print(similar)
show_at_1 = similar[0]
show_at_2 = similar[1]
show_at_3 = similar[2]

[(1.3108111500271891, '10.pdf', 'Paragraph reserved for future variants \n   \n     \n     \n   \n     \n     \n   \n \n \n     \n     \n   \nEuropean Aviation Safety Agency/Joint Aviation Authorities  \nJoint Operational Evaluation Board - Gulfstream G200 ﬁcatch-up ﬁ \nIssue 0 \n25 September 2008 \n13 10.'), (1.3108111500271891, '11.pdf', 'Paragraph reserved for future variants \n   \n     \n     \n   \n     \n     \n   \n \n \n     \n     \n   \nEuropean Aviation Safety Agency/Joint Aviation Authorities  \nJoint Operational Evaluation Board - Gulfstream G200 ﬁcatch-up ﬁ \nIssue 0 \n25 September 2008 \n13 10.'), (1.3108111500271891, '12.pdf', 'Paragraph reserved for future variants \n   \n     \n     \n   \n     \n     \n   \n \n \n     \n     \n   \nEuropean Aviation Safety Agency/Joint Aviation Authorities  \nJoint Operational Evaluation Board - Gulfstream G200 ﬁcatch-up ﬁ \nIssue 0 \n25 September 2008 \n13 10.'), (1.3108111500271891, '13.pdf', 'Paragraph reserved for future variant

In [171]:
print(show_at_1)

(1.3108111500271891, '10.pdf', 'Paragraph reserved for future variants \n   \n     \n     \n   \n     \n     \n   \n \n \n     \n     \n   \nEuropean Aviation Safety Agency/Joint Aviation Authorities  \nJoint Operational Evaluation Board - Gulfstream G200 ﬁcatch-up ﬁ \nIssue 0 \n25 September 2008 \n13 10.')


In [172]:
print(show_at_3)

(1.3108111500271891, '12.pdf', 'Paragraph reserved for future variants \n   \n     \n     \n   \n     \n     \n   \n \n \n     \n     \n   \nEuropean Aviation Safety Agency/Joint Aviation Authorities  \nJoint Operational Evaluation Board - Gulfstream G200 ﬁcatch-up ﬁ \nIssue 0 \n25 September 2008 \n13 10.')


In [38]:
from pptx import Presentation

In [46]:
# New one at the bottom
"""
#files = [x for x in os.listdir() if x.endswith(".pptx")]
files = ['2.pptx', '1.pptx', '3.pptx', '4.pptx']


for eachfile in files:
    prs = Presentation(eachfile)
    print(eachfile)
    print("----------------------")
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                print ((shape.text))
"""
        

2.pptx
----------------------
The Missed Approach –
Hey Where’s the Runway?
1
Missed Approach Segment
A Missed approach is a procedure used when an instrument approach cannot be completed to a full-stop landing
The Missed Approach Segment is that part of an instrument approach procedure between the missed approach point (MAP), the missed approach way-point (MAWP), or the point of arrival at the decision height and the specified missed approach NAVAID (navigational aid), intersection, fix, or waypoint, as appropriate, at the minimum IFR altitude
It is in this part of the approach procedure that the aircraft will climb and return to the en route structure or is positioned for holding or subsequent approach
The missed approach route and altitudes are shown on the instrument approach charts
2
The Missed Approach
The missed approach is one of the most difficult and critical phases of an instrument approach and is the one practiced the least
High workload, as the configuration of the aircraf

In [238]:
Complete1 = {}
files = ['2.pptx', '1.pptx', '3.pptx', '4.pptx']

def readPpt(data):
    data = []
    prs = Presentation(eachfile)
    #print(eachfile)
    #print("----------------------")
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                data.append(shape.text)
                
    return '\n'.join(data)

files = ['1.pptx', '2.pptx', '3.pptx', '4.pptx']
Text = []
for eachfile in files:
    data = readPpt(eachfile)      
    Text1 = (summarize(data, 10)) 
    Text.append(Text1)
    #print(Text)
    print(len(Text))
    Z = Vect(Text1, 10)
    X1 = Z[0]
    km = Z[1]
    vectorizer = Z[2]
    #Complete[str(i)+'.pdf'] = X
    Complete1[eachfile] = X1

print([len(i) for i in Text])
#print(Text[1])
print(len(Text))

Summarise Completed
1
Initialization complete
Iteration  0, inertia 7.020
Iteration  1, inertia 3.534
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
Initialization complete
Iteration  0, inertia 6.994
Iteration  1, inertia 3.991
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
 
 
Vect Completed
Summarise Completed
2
Initialization complete
Iteration  0, inertia 7.107
Iteration  1, inertia 3.693
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
Initialization complete
Iteration  0, inertia 6.948
Iteration  1, inertia 3.647
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
 
 
Vect Completed
Summarise Completed
3
Initialization complete
Iteration  0, inertia 7.030
Iteration  1, inertia 4.074
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
Initialization complete
Iteration  0, inertia 6.771
Iteration  1, inertia 3.647
Conver

In [237]:
print(len(Text[3]))

10


In [240]:
post1 = """A Missed approach is a procedure used when an instrument approach cannot be completed to a full-stop landing
The Missed Approach Segment is that part of an instrument approach procedure between the missed approach point (MAP), the missed approach way-point (MAWP), or the point of arrival at the decision height and the specified missed approach NAVAID (navigational aid), intersection, fix, or waypoint, as appropriate, at the minimum IFR altitude
"""
new_post_vec1 = vectorizer.transform([post1])
#print(new_post_vec1)
files = ['2.pptx', '1.pptx', '3.pptx', '4.pptx']
#print(Complete1)
similar1 = []
#print(Text)
for i in range(1, 5):
    name = str(i)+'.pptx'
    X1 = Complete1[str(i)+'.pptx']
    #print(Text)
    for j in range(0,10):
        try:
            dist = sp.linalg.norm((new_post_vec1 - X1[j]).toarray())
            #print(dist)
            print(j)
            similar1.append((dist, name, Text[i-1][j]))
        except Exception as e:
            print(e)
            break
similar1 = sorted(similar1) 
print(len(similar1))
print(similar1)
show_at_1 = similar1[0]
show_at_2 = similar1[1]
show_at_3 = similar1[2]

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
list index out of range
0
1
2
3
4
5
6
7
8
9
31
[(1.107274374295669, '1.pptx', 'ATC receives automated msg when pilot complete manuever\nATC: Decrease Elevation by 500meters\nATC receives automated msg when pilot complete manuever\n…\nSIB by Destination\n\n\n\n\n\n\n\n\n\n\n\n\nhttp://www.flightstats.com/go/FlightStatus/flightStatusByAirport.do\nSIB by Destination: \nAirport Witholding Procedure\n\nSIB\nSummary and Benefits\n\nSituational Awareness of Pilots\n\n Fuel savings\nDataComm has many unique advantages, but also many challenges associated\nChange in National Air System is more “evolution than revolution” (F. Ketcham)\nContinual Development of innovative ideas to further NextGen\nContinual research into NextGen enablers to improve current system\n\nConclusion\n70\n\n\nThank you!'), (1.107274374295669, '2.pptx', 'C.\nAdequate knowledge of the missed approach procedure elements for standard approaches\nInitiates the missed approach promp

In [241]:
print(show_at_1)

(1.107274374295669, '1.pptx', 'ATC receives automated msg when pilot complete manuever\nATC: Decrease Elevation by 500meters\nATC receives automated msg when pilot complete manuever\n…\nSIB by Destination\n\n\n\n\n\n\n\n\n\n\n\n\nhttp://www.flightstats.com/go/FlightStatus/flightStatusByAirport.do\nSIB by Destination: \nAirport Witholding Procedure\n\nSIB\nSummary and Benefits\n\nSituational Awareness of Pilots\n\n Fuel savings\nDataComm has many unique advantages, but also many challenges associated\nChange in National Air System is more “evolution than revolution” (F. Ketcham)\nContinual Development of innovative ideas to further NextGen\nContinual research into NextGen enablers to improve current system\n\nConclusion\n70\n\n\nThank you!')


In [242]:
print(show_at_2)

(1.107274374295669, '2.pptx', 'C.\nAdequate knowledge of the missed approach procedure elements for standard approaches\nInitiates the missed approach promptly by applying power, establishing a climb attitude, and reducing drag in accordance with the aircraft manufacturer’s recommendations\nReports beginning the missed approach procedure to ATC\nComplies with the published or alternate missed approach procedure\nAdvises ATC or examiner anytime that the aircraft is unable to comply with a clearance, restriction, or climb gradient\nFollows the recommended checklist items appropriate to the go-around procedure\nRequests, if appropriate, ATC clearance to the alternate airport, clearance limit, or as directed by the examiner\nMaintains the recommended airspeed within ±10 knots; heading, course, or bearing within ±10°; and altitude(s) within ±100 feet during the missed approach procedure.')
